In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import os
import sys
import time

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
sns.set(style="whitegrid")
import matplotlib
label_size = 9
matplotlib.rcParams['xtick.labelsize'] = label_size
matplotlib.rcParams['ytick.labelsize'] = label_size
y_formatter = matplotlib.ticker.ScalarFormatter(useOffset=False)
import matplotlib.pyplot as plt

# Project root path
PROJECT_ROOT = os.path.abspath(
    os.path.join('..', '..'))
sys.path.append(PROJECT_ROOT)

from entropy.modules.correlated_gaussians import CorrelatedGaussians
from entropy.modules.matrix_estimator import MatrixEstimator
from entropy.utils import constants

%matplotlib inline

## Visualization in 2D

In [ ]:
# Generate 2D data
n_samples = 100

data = CorrelatedGaussians()
x_samples, y_samples = data.sample(dimension=1, corr_factor=0.0, n_samples=n_samples)
z_samples = np.concatenate([x_samples, y_samples], axis=1).astype(np.float32)

fig, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=100)
ax.scatter(z_samples[:, 0], z_samples[:, 1])
ax.axis('square')
ax.set_xlim([-4, 4])
ax.set_ylim([-4, 4])
ax.set_title(
    '2D Gaussian data\n%d samples' 
    % n_samples, fontsize=10)
plt.show()

## Estimation of entropy of 2D variable

We show here the effect of the gaussian kernel width ($\sigma$) on the estimation. Note that we have the following properties for the estimator regarding $\sigma$ (proof is not shown here)

$$\lim_{\sigma\to 0} S(X) = \lim_{\sigma\to 0} I(X;Y) = \ln(N)$$
$$\lim_{\sigma\to \infty} S(X) = \lim_{\sigma\to \infty} I(X;Y) = 0$$

Where $N$ is the number of samples. We see that the estimator is bounded from below and from above. We say that the estimator is "saturated" if we reach the upper bound.

Since the estimator relies on the entropy of the discrete probability distribution defined by the $N$ eigenvalues of the normalized Gram matrix of $N\times N$ (obtained from the kernel evaluation on every pair of samples), we can interpret this result in the following way:

- If $\sigma$ is too small, the estimation converges to a maximum entropy case, which means that the values of the eigenvalues tend to be uniform. That is, each eigenvalue is equal to one another, and equal to a value of $1/N$.
- If $\sigma$ is too large, the estimation converges to a minimum entropy case, which means that the distribution tends to collapse to a "single bin". That is, a single eigenvalue has all the possible weight (equal to 1), while the rest has value of 0.

Additionally, we can interpret $\sigma$ as some sort of "bin size", if we were talking about histogram estimates.
- The larger the value of $\sigma$, the larger is the number of samples that end up in a bin, with the limiting case of all samples in one single bin (collapse). 
- The smaller the value of $\sigma$, the smaller is the number of samples that end up in a bin, with the limiting case of only one sample per bin, leaving every bin with equal size (uniformity).

We can expect that a proper tuning of $\sigma$ is needed to avoid both extremes and have meaningful estimates and comparisons between different variables. It can be proved that, without proper care, changes in variance or changes in the number of dimensions can increase or decrease the "effective" sigma, introducing artifacts.

In [ ]:
print('Saturation value of estimator log(number of samples): %1.4f' % np.log(n_samples))

In [ ]:
sigma_zero_list = [0.01, 0.1, 1, 10, 100]

fig, ax = plt.subplots(
    len(sigma_zero_list), 2, 
    figsize=(13, 2*len(sigma_zero_list)), 
    gridspec_kw = {'width_ratios':[1, 3]})

for i, sigma_zero in enumerate(sigma_zero_list):
    # Compute gram matrix
    tf.reset_default_graph()
    estimator = MatrixEstimator(sigma_zero, normalize_dimension=False, normalize_scale=False)
    norm_gram = estimator.normalized_gram(z_samples)
    with tf.Session() as sess:
        norm_gram_np = sess.run(norm_gram)

    # Compute eigenvalues
    w, _ = np.linalg.eig(norm_gram_np)
    w = np.real(w)
    w = np.clip(w, 0, None)
    w = w / w.sum()
    w = -np.sort(-w)

    alpha = 1.01  # Close to one to approximate Shannon
    entropy = np.log(np.sum(w ** alpha)) / (1.0 - alpha)

    ax[i, 0].set_title('Normalized Gram')
    ax[i, 0].imshow(norm_gram_np, vmin=0, vmax=1/n_samples, cmap='gray')
    ax[i, 0].grid(False)
    ax[i, 0].set_xticks([])
    ax[i, 0].set_yticks([])
    ax[i, 0].set_ylabel('$\sigma=%s$' % sigma_zero, fontsize=16)

    ax[i, 1].bar(np.arange(w.size), w, 1, label='Estimated Entropy %1.4f' % (entropy))
    ax[i, 1].set_ylabel('Value of Eigenvalue')
    ax[i, 1].legend()

plt.tight_layout()
plt.show()